In [ ]:
# Install required packages
%pip install torch torchinfo wandb numpy pandas tqdm tensorboard


In [ ]:
# Clone the repository and set up the environment
!git clone https://github.com/your-username/recurrent-rebels.git
%cd recurrent-rebels

# Add the project root to Python path
import sys
sys.path.append('.')


In [ ]:
# Login to Weights & Biases
import wandb
wandb.login()  # This will prompt for your API key if not already logged in


In [ ]:
# Method 1: Upload from local machine
from google.colab import files

# Create data directory
!mkdir -p data

# Uncomment and run this to upload files from your computer
# uploaded = files.upload()
# for filename in uploaded.keys():
#     !mv "{filename}" "data/{filename}"


In [ ]:
# Method 2: Download from Google Drive
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Uncomment and modify paths to copy files from Drive
# !cp /content/drive/MyDrive/path_to_data/train.npz data/
# !cp /content/drive/MyDrive/path_to_data/val.npz data/
# !cp /content/drive/MyDrive/path_to_data/train_vocab.json data/


In [ ]:
# Method 3: Download from URL
# Uncomment and modify URLs to download files
# !wget your_data_url/train.npz -O data/train.npz
# !wget your_data_url/val.npz -O data/val.npz
# !wget your_data_url/train_vocab.json -O data/train_vocab.json


In [ ]:
# Verify data files
import os

required_files = ['train.npz', 'val.npz', 'train_vocab.json']
all_files_present = True

for file in required_files:
    path = os.path.join('data', file)
    if os.path.exists(path):
        size_mb = os.path.getsize(path) / (1024 * 1024)
        print(f"✓ {file:<15} found ({size_mb:.1f} MB)")
    else:
        print(f"✗ {file:<15} missing")
        all_files_present = False

if not all_files_present:
    print("\nPlease ensure all required files are present before proceeding.")


In [ ]:
# Import the sweep configuration and training function
from hensel.sweep_config import init_sweep
from hensel.train import train

# Initialize the sweep
sweep_id = init_sweep()
print(f"Sweep initialized with ID: {sweep_id}")
print(f"\nView sweep at: https://wandb.ai/{wandb.run.entity}/hensel-model/sweeps/{sweep_id}")


In [ ]:
# Run the sweep
# You can adjust the number of trials by changing count
wandb.agent(sweep_id, function=train, count=20)  # Run 20 trials


In [ ]:
# Get the best run from the sweep
api = wandb.Api()
sweep = api.sweep(f"{wandb.run.entity}/hensel-model/{sweep_id}")
best_run = sweep.best_run()

print(f"Best run: {best_run.name}")
print(f"Best validation loss: {best_run.summary.get('best_val_loss', 'N/A')}")
print("\nBest hyperparameters:")
for key, value in best_run.config.items():
    print(f"{key}: {value}")


In [ ]:
# Download the best model
best_model_path = os.path.join('models', 'hensel', best_run.name, 'best_model.pth')
best_run.file('best_model.pth').download(replace=True)

print(f"Best model downloaded to: {best_model_path}")

# Load and verify the model
checkpoint = torch.load(best_model_path)
print("\nModel checkpoint contains:")
for key in checkpoint.keys():
    print(f"- {key}")


In [ ]:
# Plot learning curves
import matplotlib.pyplot as plt
import pandas as pd

# Get the metrics from the best run
history = pd.DataFrame(best_run.scan_history())

plt.figure(figsize=(12, 4))

# Plot training and validation loss
plt.subplot(1, 2, 1)
plt.plot(history['epoch'], history['train_loss'], label='Train')
plt.plot(history['epoch'], history['val_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Curves')
plt.legend()

# Plot R² scores
plt.subplot(1, 2, 2)
plt.plot(history['epoch'], history['r2_log'], label='R² (log scale)')
plt.plot(history['epoch'], history['r2_real'], label='R² (real scale)')
plt.xlabel('Epoch')
plt.ylabel('R²')
plt.title('R² Scores')
plt.legend()

plt.tight_layout()
plt.show()
